### Hybrid Search Langchain

In [8]:
!pip install --upgrade --quiet  pinecone pinecone-text pinecone-notebooks

In [22]:
api_key="pcsk_6nyC9J_9y3fPXWZND42uCmY7nuvzNNkbGHzZzJYzUFB2Z4ZqnGDExpa6jxS3B1sLmPJVT9"

In [23]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [24]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [25]:
index=pc.Index(index_name)
index

In [26]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [27]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()#uses tfidf by default
bm25_encoder

In [28]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<?, ?it/s]


In [29]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [30]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000018CBC7E6B90>, index=<pinecone.db_data.index.Index object at 0x0000018CD7257EB0>)

In [31]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


In [32]:
retriever.invoke("What city did i visit first")

[Document(metadata={'score': 0.239368573}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.232817784}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.212499827}, page_content='In 2023, I visited Paris')]

In [ ]:
# https://app.pinecone.io/organizations/-OUmKTjXzGjxvcw6eNY_/projects/877be4df-09a6-4d01-84a6-3685d8bd5cf0/indexes/hybrid-search-langchain-pinecone/browser